In [19]:
import pandas as pd


In [20]:
sheet_data = pd.read_excel(r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Pendencias\Arquivos\Mapa2.xlsx", sheet_name='Sheet1')

sheet_data.columns

Index(['Unnamed: 0', 'Material', 'CE04_zva70', 'CE05_zva70', 'CE07_zva70',
       'CE11_zva70', 'CE13_zva70', 'Denominação', 'Criado por', 'Canal',
       'Pendências Gerais', 'CE01_zstok', 'CE03_zstok', 'CE05_zstok',
       'CE07_zstok', 'CE09_zstok', 'CE11_zstok', 'CE13_zstok', 'Contr.qualid.',
       'Nov/24_FUP', 'Dez/24_FUP', 'Jan/24_FUP', 'Fev/24_Fup', 'Montante',
       'Modelos Lista Técnica', 'Atual', 'Similar1', 'Similar2', 'Similar3',
       'Similar4', 'Quant estoque'],
      dtype='object')

In [21]:
sheet_data['pendencia_total'] = sheet_data.filter(like='_zva70').sum(axis=1)

sheet_data['estoque_total'] = sheet_data.filter(like='_zstok').sum(axis=1)

sheet_data['FUP_total'] = sheet_data.filter(like='_FUP').sum(axis=1)

sheet_data['estoque_no_similar'] = sheet_data.apply(
    lambda row: row['Quant estoque'] - row['estoque_total'] if row['Quant estoque'] > 0 else 0,
    axis=1
)


In [22]:
sheet_data

,Unnamed: 0,Material,CE04_zva70,CE05_zva70,CE07_zva70,CE11_zva70,CE13_zva70,Denominação,Criado por,Canal,...,Atual,Similar1,Similar2,Similar3,Similar4,Quant estoque,pendencia_total,estoque_total,FUP_total,estoque_no_similar
0,0,15AG76710,0,0,0,2,0,CORREIA 2,JULIA MARIA MORETO DA SILVA,VENDA DIRETA,...,0,0,0,0,0,0,2,0,0,0
1,1,15BA77031E,0,0,0,0,13,ENGRENAGEM 27 DENTES,ROBERTA MACHADO,VENDA INDIRETA,...,0,0,0,0,0,0,13,15,0,0
2,2,20AAR70600,0,0,0,0,1,LIMPADOR,MARIANA PEREIRA DOS ANJOS,VENDA INDIRETA,...,0,0,0,0,0,0,1,2,0,0
3,3,25AA77890,0,0,0,1,0,ENGRENAGEM 36 DENTES,JOÃO PAULO CLEMENTE DOS SANTOS,NaN,...,0,0,0,0,0,0,1,1,0,0
4,4,4038258001,0,0,0,1,0,ENGRENAGEM 16D,JULIA MARIA MORETO DA SILVA,NaN,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,AD7EP72200,1,0,0,0,0,ROLO DE LIMPEZA /SP,VANESSA ARAUJO,VENDA INDIRETA,...,0,0,0,0,0,0,1,0,0,0
96,96,AD7ER72500,2,0,0,0,0,CONJUNTO DE ROLO /1,REGILENE DE CASSIA LUIZ OLIVEIRA,VENDA DIRETA,...,AD7ER72500,-,-,-,-,0,2,0,2,0
97,97,AD7ER72600,2,0,0,0,0,CONJUNTO DE ROLO /2,REGILENE DE CASSIA LUIZ OLIVEIRA,VENDA DIRETA,...,AD7ER72600,-,-,-,-,0,2,0,2,0
98,98,SP00-0220,0,0,0,0,19,TERMOSTATO INFERIOR,ROBERTA MACHADO,VENDA INDIRETA,...,0,0,0,0,0,19,19,9,0,10


In [23]:
sheet_data['projecao_compra'] = sheet_data['pendencia_total'] - (sheet_data['estoque_total'] + sheet_data['FUP_total'] + sheet_data['estoque_no_similar'])
sheet_data['projecao_compra'] = sheet_data['projecao_compra'].apply(lambda x: max(x, 0))



In [24]:
sheet_data

,Unnamed: 0,Material,CE04_zva70,CE05_zva70,CE07_zva70,CE11_zva70,CE13_zva70,Denominação,Criado por,Canal,...,Similar1,Similar2,Similar3,Similar4,Quant estoque,pendencia_total,estoque_total,FUP_total,estoque_no_similar,projecao_compra
0,0,15AG76710,0,0,0,2,0,CORREIA 2,JULIA MARIA MORETO DA SILVA,VENDA DIRETA,...,0,0,0,0,0,2,0,0,0,2
1,1,15BA77031E,0,0,0,0,13,ENGRENAGEM 27 DENTES,ROBERTA MACHADO,VENDA INDIRETA,...,0,0,0,0,0,13,15,0,0,0
2,2,20AAR70600,0,0,0,0,1,LIMPADOR,MARIANA PEREIRA DOS ANJOS,VENDA INDIRETA,...,0,0,0,0,0,1,2,0,0,0
3,3,25AA77890,0,0,0,1,0,ENGRENAGEM 36 DENTES,JOÃO PAULO CLEMENTE DOS SANTOS,NaN,...,0,0,0,0,0,1,1,0,0,0
4,4,4038258001,0,0,0,1,0,ENGRENAGEM 16D,JULIA MARIA MORETO DA SILVA,NaN,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,AD7EP72200,1,0,0,0,0,ROLO DE LIMPEZA /SP,VANESSA ARAUJO,VENDA INDIRETA,...,0,0,0,0,0,1,0,0,0,1
96,96,AD7ER72500,2,0,0,0,0,CONJUNTO DE ROLO /1,REGILENE DE CASSIA LUIZ OLIVEIRA,VENDA DIRETA,...,-,-,-,-,0,2,0,2,0,0
97,97,AD7ER72600,2,0,0,0,0,CONJUNTO DE ROLO /2,REGILENE DE CASSIA LUIZ OLIVEIRA,VENDA DIRETA,...,-,-,-,-,0,2,0,2,0,0
98,98,SP00-0220,0,0,0,0,19,TERMOSTATO INFERIOR,ROBERTA MACHADO,VENDA INDIRETA,...,0,0,0,0,19,19,9,0,10,0


In [25]:
valores_unicos = sheet_data['estoque_no_similar'].unique()
valores_unicos

array([   0,  118,   11,    4, 1220,   64,   37,   46,  189,    8,   10],
      dtype=int64)

In [26]:

caminho_saida = r'C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Pendencia\Mapa Analisado1.0.xlsx'

sheet_data.to_excel(caminho_saida, index=True)

In [27]:
def atender_estoque_ajustado_v2(row):
    if row['projecao_compra'] == 0:
        for col in ['CE05_zstok', 'CE13_zstok', 'CE11_zstok', 'CE09_zstok', 'CE07_zstok', 'CE01_zstok', 'estoque_total', 'estoque_no_similar']:
            if row[col] >= row['pendencia_total']:
                if col == 'estoque_no_similar':
                    return "Atender Pelo Similar"
                else:
                    return f"Atender pelo {col.split('_')[0]}"
        if row['FUP_total'] >= row['pendencia_total']:
            return "Aguardar IMP"
        else:
            return "Sem estoque disponível"
    else:
        return row['projecao_compra']  

sheet_data['priorizacao_estoque'] = sheet_data.apply(atender_estoque_ajustado_v2, axis=1)



In [28]:
sheet_data

,Unnamed: 0,Material,CE04_zva70,CE05_zva70,CE07_zva70,CE11_zva70,CE13_zva70,Denominação,Criado por,Canal,...,Similar2,Similar3,Similar4,Quant estoque,pendencia_total,estoque_total,FUP_total,estoque_no_similar,projecao_compra,priorizacao_estoque
0,0,15AG76710,0,0,0,2,0,CORREIA 2,JULIA MARIA MORETO DA SILVA,VENDA DIRETA,...,0,0,0,0,2,0,0,0,2,2
1,1,15BA77031E,0,0,0,0,13,ENGRENAGEM 27 DENTES,ROBERTA MACHADO,VENDA INDIRETA,...,0,0,0,0,13,15,0,0,0,Atender pelo estoque
2,2,20AAR70600,0,0,0,0,1,LIMPADOR,MARIANA PEREIRA DOS ANJOS,VENDA INDIRETA,...,0,0,0,0,1,2,0,0,0,Atender pelo CE05
3,3,25AA77890,0,0,0,1,0,ENGRENAGEM 36 DENTES,JOÃO PAULO CLEMENTE DOS SANTOS,NaN,...,0,0,0,0,1,1,0,0,0,Atender pelo CE01
4,4,4038258001,0,0,0,1,0,ENGRENAGEM 16D,JULIA MARIA MORETO DA SILVA,NaN,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,AD7EP72200,1,0,0,0,0,ROLO DE LIMPEZA /SP,VANESSA ARAUJO,VENDA INDIRETA,...,0,0,0,0,1,0,0,0,1,1
96,96,AD7ER72500,2,0,0,0,0,CONJUNTO DE ROLO /1,REGILENE DE CASSIA LUIZ OLIVEIRA,VENDA DIRETA,...,-,-,-,0,2,0,2,0,0,Aguardar IMP
97,97,AD7ER72600,2,0,0,0,0,CONJUNTO DE ROLO /2,REGILENE DE CASSIA LUIZ OLIVEIRA,VENDA DIRETA,...,-,-,-,0,2,0,2,0,0,Aguardar IMP
98,98,SP00-0220,0,0,0,0,19,TERMOSTATO INFERIOR,ROBERTA MACHADO,VENDA INDIRETA,...,0,0,0,19,19,9,0,10,0,Sem estoque disponível


In [29]:
caminho_saida = r'C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Pendencia\Arquivos\Mapa Analisado2.0.xlsx'

sheet_data.to_excel(caminho_saida, index=True)